In [1]:
using SharedArrays, HDF5

### First Method

Work with SharedArray and backup to hdf5 file.

**Cons**: An additional file is created for SharedArray.

1. Create shared array
2. Fill shared array
3. Write shared array to hdf5 file

In [8]:
# Create empty SharedArray
A = SharedArray{Float64}(abspath("sharedmmap"), (300,100), mode="w+");

# Fill SharedArray
A .= rand(1:100, size(A)...);

# Write SharedArray to HDF5File
h5open("file.h5", "w") do fid
    write(fid, "factors/P", A)
end

# Read HDF5File
B = h5read("file.h5", "factors/P")

# Remove files
rm("file.h5")
rm("sharedmmap")

B

HDF5-DIAG: Error detected in HDF5 (1.10.4) thread 0:
  #000: H5O.c line 120 in H5Oopen(): unable to open object
    major: Object header
    minor: Can't open object
  #001: H5Oint.c line 596 in H5O__open_name(): unable to open object
    major: Object header
    minor: Can't open object
  #002: H5Oint.c line 551 in H5O_open_name(): object not found
    major: Object header
    minor: Object not found
  #003: H5Gloc.c line 422 in H5G_loc_find(): can't find object
    major: Symbol table
    minor: Object not found
  #004: H5Gtraverse.c line 851 in H5G_traverse(): internal path traversal failed
    major: Symbol table
    minor: Object not found
  #005: H5Gtraverse.c line 627 in H5G__traverse_real(): traversal operator failed
    major: Symbol table
    minor: Callback failed
  #006: H5Gloc.c line 378 in H5G__loc_find_cb(): object 'P' doesn't exist
    major: Symbol table
    minor: Object not found


ErrorException: Error opening object //factors/P

### Second Method

Back shared array on hdf5 file.

**Cons**: Problems could arise with hdf5 file...

1. Create hdf5 file and dataset
2. Create shared array in realtion to hdf5 file

In [94]:
# Create h5 file and dataset
offset = h5open("file2.h5", "w") do fid
    dset = d_create(fid, "factors/P", datatype(Float64), dataspace((300,100)))
    dset[:,:] = rand(1:100, 300, 100)
    HDF5.h5d_get_offset(fid["factors/P"])
end

# build SharedArray on top of h5 file
B = SharedArray{Float64}(abspath("file2.h5"), (300,100), offset, mode="r+")

# Remove files
rm("file2.h5")

B

300×100 SharedArray{Float64,2}:
 38.0  96.0  63.0   38.0  100.0  23.0  10.0  …   3.0  64.0   98.0  48.0  95.0
 16.0  33.0  77.0    5.0   85.0  17.0  39.0     65.0  12.0   13.0  30.0  61.0
 70.0  24.0  31.0   72.0   93.0  15.0  75.0      3.0   4.0   91.0  56.0  16.0
  2.0  99.0  38.0   49.0   82.0  35.0  85.0     95.0  41.0   71.0  79.0  71.0
 35.0  25.0  19.0   64.0    6.0  56.0  51.0     35.0   6.0   23.0  80.0  77.0
 80.0  22.0  12.0    4.0   50.0  71.0  82.0  …  74.0  12.0    4.0   3.0  19.0
 15.0  70.0  71.0   33.0   81.0  23.0  41.0     32.0  84.0   83.0   8.0  42.0
 53.0  66.0  76.0   52.0   14.0  68.0  24.0     41.0  35.0   17.0  12.0  56.0
 95.0  83.0  64.0   57.0   97.0  11.0   2.0     25.0   5.0   83.0  51.0  58.0
 38.0  63.0  17.0   42.0   42.0  38.0  12.0     98.0   9.0   96.0  71.0  98.0
 62.0  19.0  65.0  100.0    8.0  63.0  48.0  …   6.0   2.0   97.0  38.0  12.0
 59.0  45.0  67.0   66.0   53.0  42.0  16.0     37.0  26.0   40.0  83.0  32.0
 55.0  37.0  69.0   14.0   90.0 